In [ ]:
# Import and test pyvirtualdisplay
!pip3 install pyvirtualdisplay
!export DISPLAY=:0
import os
os.environ['PYVIRTUALDISPLAY_DISPLAYFD'] = '0'
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
#######################################
# INSTALL AND IMPORT DEPENDENCIES
#######################################

!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
#######################################
# CREATE ENVIRONMENT
#######################################

env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())

In [ ]:
#######################################
# VECTORIZE AND NORMALISE ENVIRONMENT
#######################################

env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=5.)

In [ ]:
v#######################################
# CREATE MODEL
#######################################

model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1,
            device="cuda")

In [ ]:
#######################################
# TRAIN MODEL
#######################################

model.learn(1_000_000)

In [ ]:
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

In [ ]:
#######################################
# EVALUATE MODEL
#######################################

from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
#######################################
# PUSH TO HUB
#######################################

notebook_login()

In [ ]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"mattbailey1991/a2c-{env_id}", # Change the username
    commit_message="Initial commit",
)